


# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Autograd Basics and NN Module

### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P19A06E_test" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "981234567" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="BLR_M2W7_DEMO_EXP2" #name of the notebook

def setup():
    ipython.magic("sx pip3 install torch")
   
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
import torch

###Creating a random tensor

In [0]:
x = torch.rand(3,3)
print(x)
print(x.type())

tensor([[0.1768, 0.9096, 0.6366],
        [0.0256, 0.1732, 0.7924],
        [0.2874, 0.4250, 0.3732]])
torch.FloatTensor


###checking the requires_grad flag which tells whether the tensor support gradient calculation
###By default it is false

In [0]:
print(x.requires_grad)

False


**Autograd-1** (Automatic Differentiation) 

Example 1: <br>
Consider the following analytical function:- <br>
$y=4x^2\\
z=3y+5\\
l=\frac{1}{4} \sum_{i}^4{z_i}\\
$
<br>
Let, $x = \left[ \begin{matrix}
  1 & 1 \\
  1 & 1
 \end{matrix}\right ]$
 <br>
The above set of functions can be visualized as a **computational graph**.

###Initialiation of x (notice we enforced the computation of gradients)

In [0]:
x = torch.ones(2,2,requires_grad=True)
print(x)
print(x.type())

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
torch.FloatTensor


###Creating the computational graph
###The graph is created in a dynamic manner (on the fly)

In [0]:
y = 4*x*x
z = 3*y+5
z.retain_grad() #Only used if we want to preserve the gradients in intermediate variables. (Generally not retained)
out = z.mean()
print(out)

tensor(17., grad_fn=<MeanBackward1>)


###Edges in the graph are connected with grad_fn function

In [0]:
print(z.grad_fn)

###Calling the backward operation to compute gradients of "out" variable w.r.t input tensor

In [0]:
out.backward()

In [0]:
print(x.grad)

tensor([[6., 6.],
        [6., 6.]])


#**Autograd-1** 
###Example 2

###Consider the following linear function (line equation)
###Here w and b can be treated as parameters of a one layer neural network.

In [0]:
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

y = w * x + b    #e.g. y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


# **Autograd-2 with nn.Module**

###Designing a simple (one hidden layer) neural network using torch.nn module 

* torch.nn is a specialized package for neural network functions
* torch.optim provides various inbuilt optimization routines for training neural network

In [0]:
import torch.nn as nn
import torch.optim as optim

###Create a random input x and expected output y 

In [0]:
x = torch.rand(3)
y = torch.tensor(1.0)

###Creating a Linear layer using nn Module with weight and bias paramters (just like example 2 in Autograd-1)

* weights (initialized randomly) but notice by default the grad is enabled


In [0]:
fc1 = nn.Linear(3,1)
print("x: \n\n ", x)


print("Weights: \n\n ", fc1.weight)

print("Bias: \n\n ", fc1.bias)


x: 

  tensor([0.6626, 0.5846, 0.2830])
Weights: 

  Parameter containing:
tensor([[-0.3329,  0.2321, -0.4230]], requires_grad=True)
Bias: 

  Parameter containing:
tensor([0.2266], requires_grad=True)


In [0]:
print(fc1.weight.requires_grad)

True


###Printing all the paramters given by the linear layer

In [0]:
for param in fc1.parameters():
  print(param)

Parameter containing:
tensor([[ 0.4188, -0.5132, -0.5575]], requires_grad=True)
Parameter containing:
tensor([-0.0243], requires_grad=True)


###Defining the loss function again from nn Module
###Here we use mean square loss function

In [0]:
criterion=nn.MSELoss()

###Defining the optimization function using torch.optim package.
###Here the first param is the "list of parameters" which we want to optimize and "lr" is the learning rate

In [0]:
optimizer = optim.SGD(fc1.parameters(),lr=0.1)

###Clears the gradients of parameters

In [0]:
optimizer.zero_grad()

###performing the forward pass

In [0]:
y_pred = fc1(x)
print(y_pred)

tensor([-0.5580], grad_fn=<ThAddBackward>)


###Computing loss (predicted vs actual)

In [0]:
loss = criterion(y_pred,y)
print(loss)

tensor(2.4275, grad_fn=<MseLossBackward>)


###Since loss is a scalar value, we use item() for getting the python number.

In [0]:
print(loss.item())

2.4275097846984863


###Compute gradients using backpropagation

In [0]:
loss.backward()

In [0]:
print(fc1.weight.grad)

tensor([[-2.6357, -2.4555, -2.7029]])


In [0]:
fc1.weight

Parameter containing:
tensor([[ 0.4188, -0.5132, -0.5575]], requires_grad=True)

###Performing a single step of gradient descent. Note this will change the parameters (weights and bias)

In [0]:
optimizer.step()

###Notice the change of values after the step()

In [0]:
fc1.weight

Parameter containing:
tensor([[ 0.6824, -0.2676, -0.2872]], requires_grad=True)

###Lets do a forward pass again and print the loss. 
###Note after the weight updation, the loss on the same input should decrease.

In [0]:
y_pred = fc1(x)
print(y_pred)
loss = criterion(y_pred,y)
print(loss.item())

tensor([0.4044], grad_fn=<ThAddBackward>)
0.3546842634677887


### Please answer the questions below to complete the experiment:




In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful. Ref: 9450
